Перед запуском в Google Colab необходимо сменить среду выполнения (если она уже не выбрана по-умолчанию):

Runtime -> Change runtime type -> Hardware accelerator -> T4 GPU

In [1]:
!pip install faster-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 125.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.1 MB/s eta 0:00:00


In [2]:
# загрузка аудиофайла с Google-диска

import requests
import os

current_dir = os.path.abspath('')
parent_dir = os.path.dirname(current_dir)
audio_path = os.path.join('current_dir','audio.wav')


def download_file_from_cloud_storage(id, destination):
    url = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    params = { 'id': id, 'confirm': 1}
    response = session.get(url, params = params, stream = True)

    save_response_content(response, destination)


def save_response_content(response, destination):
    chunk_size = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(chunk_size):
            if chunk:
                f.write(chunk)

download_file_from_cloud_storage('1EKCiMifsnWPRWAsvfYSYfBcrxnCkkxCK',
                                 audio_path)

In [ ]:
!apt install libcublas11

In [3]:
from faster_whisper import WhisperModel
model_size = "large-v2"
model = WhisperModel(model_size, device="cuda", compute_type="float16")
segments, info = model.transcribe(audio_path, beam_size=5)

In [4]:
print("Detected language '%s' with "
      "probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

Detected language 'ru' with probability 0.992676
[0.00s -> 8.00s]  Мы с вами еще не знакомы, так как первый день я представлю себе Менгазова Зиляра Ильдаровна.
[8.00s -> 14.00s]  Меня зовут. Я являюсь геологом Центра оценок и анализа проектов.
[14.00s -> 22.00s]  И следующие три дня я постараюсь продолжить ваше погружение в геологию.
[22.00s -> 29.00s]  И заранее хочу сказать, что если у вас есть вопросы, вы можете сразу меня прерывать по ходу введения слайдов.
[29.00s -> 36.00s]  И, соответственно, задавать любые вопросы, которые, если вам что-то осталось непонятно.
[36.00s -> 46.00s]  Коллеги, подскажите, вам нормально все видно, слышно, трансляция идет?
[46.00s -> 47.00s]  Да, да, сверху.
[47.00s -> 55.00s]  Спасибо. Тогда лучше голосом можете, пожалуйста, вопросы тоже голосом, потому что в чате у меня не будет возможности посмотреть.
[55.00s -> 59.00s]  Лучше прям прерывайте по ходу пьесы.
[59.00s -> 72.00s]  Итак, прошлое три дня, получается, вы дошли до залежей нефти и газа немно